In [19]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
import sklearn.model_selection as ms
import numpy as np
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import *
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import accuracy_score
import plotly.express as px

In [20]:
rbData = pd.read_parquet("rbDataV2.parquet")
wrData = pd.read_parquet("wrDataV2.parquet")
teData = pd.read_parquet("teDataV2.parquet")

In [21]:
rbData['position'] = [0 for i in range(rbData.shape[0])]
wrData['position'] = [1 for i in range(wrData.shape[0])]
teData['position'] = [2 for i in range(teData.shape[0])]

In [22]:
df = pd.concat([rbData, wrData, teData])

In [63]:
tier = []
thresholds = [24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, df['fantasy_points_ppr_per_game_future'].min()]
#thresholds = [6, 12, 18, 24, 30, 36, 42, 48, 60, 72, 84, 96, 108, df['fantasy_rank_future'].max()]
for row in df.itertuples():
    myBool = False
    for i in range(len(thresholds)):
        #if row.fantasy_points_ppr_per_game_future >= thresholds[i]:
        if row.fantasy_points_ppr_per_game_future >= thresholds[i]:
            myBool = True
            tier.append(i)
            break
    if not myBool:
        tier.append(None)
df['tier_future'] = tier
nextYear = df[df.season == 2021].copy()
df = df.dropna(subset=['fantasy_points_ppr_future'])

In [64]:
colsToDrop = ['player_id', 'season',
              'fantasy_points_ppr_future', 
              'fantasy_points_ppr_per_game_future',
              'fantasy_rank_future',
              'per_game_fantasy_rank_future',
              'tier_future']
# separate featues and labels
features = df.drop(colsToDrop, axis=1)
labels = df['tier_future']

In [65]:
# fit model no training data
model = xgb.XGBRegressor(objective='reg:squarederror')

In [66]:
# define model evaluation method
cv = ms.StratifiedKFold(n_splits=10)

In [67]:
# evaluate model
scores = ms.cross_val_score(model, features, labels, scoring='neg_median_absolute_error', cv=cv, n_jobs=-1)

/Users/benabraham/opt/anaconda3/envs/FantFootballV2/lib/python3.9/site-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  warnings.warn(


In [61]:
print('Mean MAE: %.3f (%.3f)' % (scores.mean(), scores.std()))

Mean MAE: -2.363 (0.279)


Previous: 0.315, 0.048

In [62]:
scores

array([-3.00062728, -2.16299248, -2.30725288, -2.40745544, -2.34242344,
       -2.59416771, -2.27223492, -2.35998344, -1.85053444, -2.32862663])

In [70]:
df[df.tier_future == 0]

,player_id,season,age,height,weight,experience,draft_pick,games,carries,rushing_yards,...,receiving_tds_per_game_n-2,receiving_air_yards_per_game_n-2,receiving_yards_after_catch_per_game_n-2,receiving_first_downs_per_game_n-2,receiving_epa_per_game_n-2,fantasy_points_ppr_per_game_n-2,fantasy_rank_n-2,per_game_fantasy_rank_n-2,position,tier_future
1,00-0026184,2013,28,74.0,218.0,5.0,13.0,16,289,1339.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
5,00-0028009,2013,25,73.0,220.0,2.0,7.0,14,217,1121.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
6,00-0025399,2013,27,71.0,215.0,6.0,12.0,16,301,1257.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
9,00-0030496,2013,21,73.0,225.0,0.0,16.0,13,244,860.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
23,00-0025394,2014,29,73.0,220.0,7.0,7.0,1,21,75.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,00-0033757,2019,25,77.0,237.0,2.0,NaN,9,0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0
0,00-0030506,2020,31,77.0,260.0,7.0,1.0,15,0,0.0,...,0.6250,85.9375,35.500000,4.250000,4.386287,18.412500,1.0,1.0,2,0
2,00-0033288,2020,27,76.0,250.0,3.0,2.0,8,2,17.0,...,0.3125,62.2500,54.375000,3.750000,4.077783,16.168750,3.0,3.0,2,0
3,00-0034753,2020,25,76.0,256.0,2.0,22.0,14,0,0.0,...,0.1875,33.1875,12.062500,1.750000,2.422514,6.700000,18.0,26.0,2,0
